In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%%capture
!pip install bertopic

In [5]:
import re
import pandas as pd
from datetime import datetime



In [6]:
# Load data
ny_post = pd.read_csv('/content/gdrive/MyDrive/Cendie/ny_post(1).csv')


In [ ]:
ny_post.Fecha

0      2020-03-01 00:00:00
1      2020-03-01 00:00:00
2      2020-03-01 00:00:00
3      2020-03-01 00:00:00
4      2020-03-01 00:00:00
              ...         
646    2020-10-25 00:00:00
647    2020-10-25 00:00:00
648    2020-10-25 00:00:00
649    2020-10-25 00:00:00
650    2020-10-25 00:00:00
Name: Fecha, Length: 651, dtype: object

In [7]:
ny_post.Texto = ny_post.apply(lambda row: re.sub(r"http\S+", "", row.Texto).lower(), 1)
ny_post.Texto= ny_post.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.Texto.split())), 1)
ny_post.Texto = ny_post.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.Texto).split()), 1)
ny_post_texto = ny_post.Texto.to_list()

In [8]:
nypost_texto= []
for i in range(len(ny_post_texto)):
  a = ny_post_texto[i].replace("sign up for our special edition newsletter to get a daily update on the coronavirus pandemic ", "") 
  nypost_texto.append(a)

In [ ]:
nypost_texto

['new york city officials are urging folks not to panic over the first confirmed case of the coronavirus in the state on sunday shortly after the big apple case was announced mayor bill de blasio released a statement saying authorities are fully prepared to respond to the development from the beginning we have said it was a matter of when not if there would be a positive case of coronavirus in new york the mayor said our health authorities have been in a state of high alert for weeks and are fully prepared to respond the statement said we will continue to ensure new yorkers have the facts and resources they need to protect themselves new york city council speaker corey johnson tweeted that this is no cause to panic the council will hold a hearing on thursday to find out how the big apple is prepared for the outbreak johnson said governor andrew cuomo on sunday evening announced the first case of the deadly virus in the state a woman in her late s who recently traveled to iran she is be

In [9]:
timestamps = ny_post.Fecha.to_list()


In [10]:
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

#text = "Nick likes to play football, however he is not too fond of tennis."
nypost_text= []

for i in range(len(nypost_texto)):
  text_tokens = word_tokenize(nypost_texto[i])
  nypost_text.append(text_tokens)

#tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]

print(nypost_text)


Output hidden; open in https://colab.research.google.com to view.

In [11]:
nypost_texto= []

for i in range(len(nypost_text)):
  text_tokens = [word for word in nypost_text[i] if not word in stopwords.words()]
  nypost_texto.append(text_tokens)


In [17]:
#nypost_texto

In [27]:
nypost_text = pd.Series(nypost_texto)
 


In [28]:
nypost_text = nypost_text.apply(lambda x: ', '.join([str(i) for i in x]))

In [30]:
import re
regex_dict = {'punct': re.compile(r'[^@#\w_]')}
   

In [31]:
nypost_text = nypost_text.str.replace(regex_dict['punct'], ' ')

In [32]:
nypost_text

0      new  york  city  officials  urging  folks  pan...
1      even  big  apple  beloved  chinese  restaurant...
2      coronavirus  claimed  second  life  us  died  ...
3      first  case  coronavirus  new  york  city  con...
4      syracuse  university  student  eli  schwemler ...
                             ...                        
646    white  house  chief  staff  mark  meadows  sun...
647    retired  usually  seasonal  retail  work  extr...
648    white  house  chief  staff  mark  meadows  sai...
649    nfl  finally  punished  tennessee  titans  tea...
650    florida  nurse  claimed  doctor  intentionally...
Length: 651, dtype: object

In [37]:
from bertopic import BERTopic
topic_model = BERTopic(language="english", min_topic_size=10, nr_topics=15 , verbose=True)
topics, _ = topic_model.fit_transform(nypost_text)

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

2021-08-21 00:16:03,833 - BERTopic - Transformed documents to Embeddings
2021-08-21 00:16:08,733 - BERTopic - Reduced dimensionality with UMAP
2021-08-21 00:16:08,777 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2021-08-21 00:16:10,499 - BERTopic - Reduced number of topics from 19 to 16


In [38]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name
0,-1,217,-1_coronavirus_virus_covid_health
1,0,102,0_city_york_coronavirus_deaths
2,1,60,1_hospital_patients_virus_coronavirus
3,2,54,2_biden_coronavirus_campaign_vaccine
4,3,38,3_restaurant_restaurants_chefs_employees
5,4,24,4_season_yankees_mets_game
6,5,23,5_nfl_game_week_players
7,6,21,6_bar_deputies_brooklyn_party
8,7,20,7_students_school_campus_schools
9,8,17,8_masks_mask_coronavirus_people


In [39]:
topic_nr = freq.iloc[2]["Topic"]  # We select a frequent topic
topic_model.get_topic(topic_nr)   # You can select a topic number as shown above

[('hospital', 0.03290130875853701),
 ('patients', 0.024037762132561154),
 ('virus', 0.021764144734353164),
 ('coronavirus', 0.019951182962299953),
 ('medical', 0.015571582495491435),
 ('patient', 0.014487033418833018),
 ('plasma', 0.010707117476613777),
 ('doctors', 0.01043242668096022),
 ('symptoms', 0.009998930637134123),
 ('died', 0.009957853219421995)]

In [40]:
fig = topic_model.visualize_topics(); fig

In [41]:
topics_over_time = topic_model.topics_over_time(docs=nypost_text, 
                                                topics=topics, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

20it [00:07,  2.75it/s]


In [43]:
topic_model.visualize_topics_over_time(topics_over_time)